<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [2]:
#Start Here

import pandas as pd

df   = pd.read_csv('./data/amazon_reviews.csv')

In [3]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore



In [5]:
gensim.__version__

'3.8.1'

In [7]:
STOPWORDS = set(STOPWORDS)

In [8]:
def tokenize(text):
    return [token for token in simple_preprocess(text, max_len=40) if token not in STOPWORDS]

In [9]:
ex_str = "This is an example of Gensim                 simple_preprocess utility function. "
tokenize(ex_str)

['example', 'gensim', 'simple_preprocess', 'utility', 'function']

In [10]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

CUSTOM_FILTERS = [lambda x: x.lower(),
                  strip_multiple_whitespaces,
                  strip_numeric,
                  remove_stopwords,
                  strip_non_alphanum,
                  strip_short
                 ]
preprocess_string(ex_str, CUSTOM_FILTERS)

['example', 'gensim', 'simple_preprocess', 'utility', 'function']

In [20]:
df['tokenize_text'] = df['reviews.text'].apply(preprocess_string,CUSTOM_FILTERS)

In [21]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokenize_text
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, qualiti, miss, backup, spri..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expens, wai, product, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracel, price, happi]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteri, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteri, long, last, price, great]"


In [24]:
id2word = corpora.Dictionary(df['tokenize_text'])

In [27]:
id2word.token2id['duracel']

16

In [28]:
id2word.doc2bow(tokenize("This is a sample message of great duracell batteries"))

[(21, 1)]

In [30]:
import sys
print(sys.getsizeof(id2word))
#print(sys.getsizeof(tokens)) 

56


In [31]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [32]:
corpus = [id2word.doc2bow(text) for text in df['tokenize_text']]

In [33]:
lda = LdaMulticore(corpus = corpus,
                  id2word=id2word,
                  random_state=42,
                  num_topics=15,
                  passes=10,
                  workers=4)

In [36]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [37]:
topics = [' '.join(t[0:10]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
love old year bought kid gift daughter christma game great

------ Topic 1 ------
sound music alexa speaker tap echo portabl good amazon wifi

------ Topic 2 ------
charg expect camera better batteri faster power charger hold like

------ Topic 3 ------
batteri work fine us remot need box week pack replac

------ Topic 4 ------
bui best recommend product amazon purchas great happi highli thank

------ Topic 5 ------
read kindl screen book love easi light tablet us size

------ Topic 6 ------
batteri good price brand long great work bui amazon life

------ Topic 7 ------
game tablet love year learn old son case bought time

------ Topic 8 ------
kindl book purchas us read easi need reader upgrad ipad

------ Topic 9 ------
tablet us easi kid good great perfect love want like

------ Topic 10 ------
great price work tablet product good love valu qualiti deal

------ Topic 11 ------
kid app set amazon friendli user tablet limit time littl

------ Topic 12 ------
tabl

In [39]:
import pyLDAvis.gensim

In [40]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.255271 -0.063222       1        1  14.166245
12    -0.058091 -0.002347       2        1   9.399368
0     -0.231832  0.008039       3        1   8.877652
5     -0.174456  0.033285       4        1   8.729216
13     0.121608  0.166637       5        1   7.272182
3      0.187351  0.149666       6        1   7.190731
9     -0.167886 -0.070935       7        1   6.164680
7     -0.179447  0.092323       8        1   5.935389
10    -0.019091 -0.285661       9        1   5.914670
8     -0.080359  0.079835      10        1   5.537712
4      0.163609 -0.110873      11        1   4.884190
11    -0.073619  0.053357      12        1   4.441823
2      0.152632  0.156029      13        1   4.366128
1     -0.005946 -0.016939      14        1   3.736957
14     0.110256 -0.189192      15        1   3.383057, topic_info=           Term         Freq        Total Category  logprob  loglift
2       batteri  9478.000000  9478.000000  Default  30.0000  30.0000
20        great  8819.000000  8819.000000  Default  29.0000  29.0000
1855     tablet  7647.000000  7647.000000  Default  28.0000  28.0000
17        price  5095.000000  5095.000000  Default  27.0000  27.0000
33         good  5791.000000  5791.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
142   recommend    88.619186  1333.052490  Topic15  -4.7623   0.6755
37          lot    83.161423  1306.747070  Topic15  -4.8258   0.6319
20        great    95.161057  8819.674805  Topic15  -4.6910  -1.1428
387     quickli    67.702606   326.901978  Topic15  -5.0315   1.8119
9          work    76.699348  4692.777832  Topic15  -4.9067  -0.7275

[1111 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
54        1  0.480848      aaa
54        5  0.008387      aaa
54        6  0.506009      aaa
2346      8  0.953332      abc
808       1  0.017247  absolut
...     ...       ...      ...
1704     14  0.023090   youtub
1006      3  0.790477       yr
1006     13  0.184445       yr
951       1  0.937765      yup
2060     13  0.911001   zipper

[3579 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 13, 1, 6, 14, 4, 10, 8, 11, 9, 5, 12, 3, 2, 15])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling